In [ ]:

from langchain.vectorstores import FAISS
from langchain.schema import Document


In [10]:
from langchain.document_loaders import PyMuPDFLoader
# Load the PDF
loader = PyMuPDFLoader("data/RAMAYANA.pdf")
docs = loader.load()

# Each doc now contains `page_content` and `metadata`
for doc in docs:
    print(doc.page_content[:50])  # Preview text
    print(doc.metadata)            # {'source': 'data/ramayana.pdf', 'page': 0}

SRI RAMA JAYAM 
RAMAYANA FOR CHILDREN 
Compiled by
{'producer': 'Microsoft® Word 2010', 'creator': 'Microsoft® Word 2010', 'creationdate': '2013-04-14T19:39:50-07:00', 'source': 'data/RAMAYANA.pdf', 'file_path': 'data/RAMAYANA.pdf', 'total_pages': 45, 'format': 'PDF 1.5', 'title': 'RAMAYANA FOR CHILDREN', 'author': 'Sony', 'subject': 'Compiled by', 'keywords': '', 'moddate': '2013-04-14T19:39:50-07:00', 'trapped': '', 'modDate': "D:20130414193950-07'00'", 'creationDate': "D:20130414193950-07'00'", 'page': 0}
1 
 
Contents 
 
1 RAMAYANA FOR CHILDREN .........
{'producer': 'Microsoft® Word 2010', 'creator': 'Microsoft® Word 2010', 'creationdate': '2013-04-14T19:39:50-07:00', 'source': 'data/RAMAYANA.pdf', 'file_path': 'data/RAMAYANA.pdf', 'total_pages': 45, 'format': 'PDF 1.5', 'title': 'RAMAYANA FOR CHILDREN', 'author': 'Sony', 'subject': 'Compiled by', 'keywords': '', 'moddate': '2013-04-14T19:39:50-07:00', 'trapped': '', 'modDate': "D:20130414193950-07'00'", 'creationDate': "D:2013041

In [11]:
# Paragraph-level splitting
def split_into_paragraphs(text):
    paragraphs = text.split("\n\n")
    return [Document(page_content=p.strip()) for p in paragraphs if p.strip()]

documents = []
for doc in docs:
    paragraphs = split_into_paragraphs(doc.page_content)
    documents.extend(paragraphs)

In [12]:
from langchain.embeddings import HuggingFaceEmbeddings

# Embedder and vectorstore
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2",
                                        model_kwargs={"device": "cpu"})

vectorstore = FAISS.from_documents(documents, embedding_model)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})


In [13]:
from langchain.prompts import PromptTemplate

story_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are a Ramayana expert who tells stories simply and accurately.

Use the context below to answer the question as a short story.

If unsure or if context is missing, say "I don't know based on the text."

Context:
{context}

Question:
{question}

Answer in story form:
"""
)


In [16]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.schema import Document

llm = ChatOpenAI(temperature=0)

def answer_with_verification(question: str):
    # Step 1: Retrieve documents
    relevant_docs = retriever.get_relevant_documents(question)
    context = "\n".join([doc.page_content for doc in relevant_docs])

    # Step 2: Generate an answer from the context
    chain = LLMChain(llm=llm, prompt=story_prompt)
    generated_answer = chain.run(context=context, question=question)

    # Step 3: Ask the LLM to verify answer correctness
    verification_prompt = f"""
    Context:
    {context}

    Generated Answer:
    {generated_answer}

    Does the generated answer contain any factual errors based on the context? Respond only with "Yes" or "No" and then explain why.
    """
    verification_response = llm.predict(verification_prompt)

    # Step 4: Final Output
    if "No" in verification_response:
        return generated_answer, "✅ The answer is correct and based on the context."
    else:
        return generated_answer, "⚠️ This answer may be hallucinated or partially incorrect.\n\n" + verification_response


In [17]:
question = "How was Lord Rama born and who were his parents?"
answer, verification = answer_with_verification(question)
print("Answer:", answer)
print("Verification:", verification)


Answer: Lord Rama was born to King Dasaratha and Queen Kaushalya of Ayodhya. King Dasaratha had three wives - Kaushalya, Kaikeyi, and Sumitra. Lord Rama was the eldest son of King Dasaratha and Queen Kaushalya. He was born as a divine incarnation of Lord Vishnu to rid the world of evil and establish righteousness. Lord Rama was known for his virtues, valiance, and devotion to his parents. His birth and life are celebrated in the epic Ramayana, written by Maharishi Valmiki.
Verification: ✅ The answer is correct and based on the context.
